In [ ]:
import requests
import re
import os
import time

In [ ]:
print(os.getcwd())

In [ ]:
# judge if the folder exist. 判断图集的文件夹是否存在，如果不存在则建立一个
def fJudgeFileNetPhos(vSeqName):    
    global vFoldNameNetPhos
    vFoldNameNetPhos = os.getcwd() + "\\" + vSeqName + "\\NetPhos磷酸化位点\\"
    os.makedirs(vFoldNameNetPhos, exist_ok = True)
    os.chdir(vFoldNameNetPhos)
    os.makedirs("Pic", exist_ok = True)

In [ ]:
def fGetNetPhos(vSeq):
    vUrl = "http://www.cbs.dtu.dk/cgi-bin/webface2.fcgi"
    vPostData ={"configfile": "/usr/opt/www/pub/CBS/services/NetPhos-3.1/NetPhos.cf",
                "SEQPASTE": vSeq,                
                "graphics": "ps"}
    r = requests.post(vUrl, data = vPostData)
    time.sleep(5)
    vRealUrl = re.search("a href=\"([\s\S]*?)\"", r.text, re.S).group(1)
    time.sleep(5)
    r = requests.get(vRealUrl)    
    return(r.text)

In [ ]:
def fSaveNetPhosPic(vHtml):
    global vCode    
    vCode = str(re.search("services/NetPhos-3.1/tmp/(.*?)/netphos-3.1b.Sequence.gif", vHtml, re.S).group(1))                           
    vPicA = "http://www.cbs.dtu.dk/services/NetPhos-3.1/tmp/" + vCode + "/netphos-3.1b.Sequence.gif"
    with open("Pic\\NetPhos.gif", "wb") as vPic1:
        r = requests.get(vPicA)
        vPic1.write(r.content)

In [ ]:
def fSoupNetPhos(vHtml):
    vNetPhosText = vHtml
    vNetPhos1 = vNetPhosText.replace(re.search("[\s\S]*?\</h2\>", vNetPhosText, re.S).group(0), "<h1>NetPhos磷酸化位点分析</h1><p><a href = \"http://www.cbs.dtu.dk/services/NetPhos/\">使用NetPhos 1.0查询</a></p>")
    vNetPhos2 = vNetPhos1.replace(re.search("Sequence.gif\"\>([\s\S]*)", vNetPhos1, re.S).group(1), "</p><h3>文件下载</h3><p><a href=\"file:///" + vFoldNameNetPhos + "Pic\\NetPhos.gif\">gif图片</a><p><a href=\"file:///" + vFoldNameNetPhos + "Pic\\NetPhos.eps\">eps图片</a></p>")
    vNetPhos3 = vNetPhos2.replace("     Asn-Xaa-Ser/Thr sequons in the sequence output below are highlighted in <font color=\"#0000ff\"><b>blue</b></font>", "Asn Xaa Ser/Thr使用高亮的<font color=\"#0000ff\"><b>蓝色</b></font>")
    vNetPhos4 = vNetPhos3.replace("/services/NetPhos-3.1/tmp/" + vCode + "//netphos-3.1b.Sequence.gif", "file:///" + vFoldNameNetPhos + "Pic\\NetPhos.gif")
    vNetPhos5 = vNetPhos4.replace(" amino acids", "个氨基酸残基").replace(" prediction results", "预测结果")
    vNetPhos6 = vNetPhos5.replace("          Asparagines predicted to be N-glycosylated are highlighted in <font color=\"#ff0000\"><b>red</font>", "N-糖基化的天冬酰胺使用高亮的<font color=\"#ff0000\"><b>红色</font>")
    vNetPhos7 = vNetPhos6.replace("Technical University of Denmark", "丹麦科技大学")
    vNetPhos = vNetPhos7
    with open("NetPhos.html", "w") as f:
        f.write(vNetPhos)

In [ ]:
def main(vSeqName, vSeq):
    print("===开始NetPhos糖基化位点分析===")
    fJudgeFileNetPhos(vSeqName)
    print("---------已创建存储目录---------")
    print("----------开始提交数据----------")
    vHtml = fGetNetPhos(vSeq)
    print("---------已获取分析结果---------")
    print("----------开始保存数据----------")
    fSaveNetPhosPic(vHtml)
    print("----------开始保存报告----------")
    fSoupNetPhos(vHtml)
    print("---------结果已保存完毕---------")

In [ ]:
with open("name.txt", "r") as f:
    vSeqName = f.read()
with open("orf.txt", "r") as f:
    vSeq = f.read()
main(vSeqName, vSeq)